In [1]:
import numpy as np
import pandas as pd

# Loading data

In [2]:
bikes = pd.read_excel("../Rscripts/data/bike_sales/bikes.xlsx")
bikes.head()

,bike.id,model,description,price
0,1,Supersix Evo Black Inc.,Road - Elite Road - Carbon,12790
1,2,Supersix Evo Hi-Mod Team,Road - Elite Road - Carbon,10660
2,3,Supersix Evo Hi-Mod Dura Ace 1,Road - Elite Road - Carbon,7990
3,4,Supersix Evo Hi-Mod Dura Ace 2,Road - Elite Road - Carbon,5330
4,5,Supersix Evo Hi-Mod Utegra,Road - Elite Road - Carbon,4260


In [3]:
bikeshops = pd.read_excel("../Rscripts/data/bike_sales/bikeshops.xlsx")
bikeshops.head()

,bikeshop.id,bikeshop.name,location
0,1,Pittsburgh Mountain Machines,"Pittsburgh, PA"
1,2,Ithaca Mountain Climbers,"Ithaca, NY"
2,3,Columbus Race Equipment,"Columbus, OH"
3,4,Detroit Cycles,"Detroit, MI"
4,5,Cincinnati Speed,"Cincinnati, OH"


In [4]:
orderlines = pd.read_excel("../Rscripts/data/bike_sales/orderlines.xlsx")
orderlines.head()

,Unnamed: 0,order.id,order.line,order.date,customer.id,product.id,quantity
0,1,1,1,2011-01-07,2,48,1
1,2,1,2,2011-01-07,2,52,1
2,3,2,1,2011-01-10,10,76,1
3,4,2,2,2011-01-10,10,52,1
4,5,3,1,2011-01-10,6,2,1


# Joining Data

In [5]:
bike_orderlines_joined = orderlines.merge(bikes, how='left',
                                          left_on="product.id", 
                                          right_on="bike.id") \
                                   .drop('bike.id', axis = 1) \
                                   .merge(bikeshops, how='left', 
                                          left_on="customer.id", 
                                          right_on="bikeshop.id") \
                                   .drop('bikeshop.id', axis=1)

In [6]:
bike_orderlines_joined.head()

,Unnamed: 0,order.id,order.line,order.date,customer.id,product.id,quantity,model,description,price,bikeshop.name,location
0,1,1,1,2011-01-07,2,48,1,Jekyll Carbon 2,Mountain - Over Mountain - Carbon,6070,Ithaca Mountain Climbers,"Ithaca, NY"
1,2,1,2,2011-01-07,2,52,1,Trigger Carbon 2,Mountain - Over Mountain - Carbon,5970,Ithaca Mountain Climbers,"Ithaca, NY"
2,3,2,1,2011-01-10,10,76,1,Beast of the East 1,Mountain - Trail - Aluminum,2770,Kansas City 29ers,"Kansas City, KS"
3,4,2,2,2011-01-10,10,52,1,Trigger Carbon 2,Mountain - Over Mountain - Carbon,5970,Kansas City 29ers,"Kansas City, KS"
4,5,3,1,2011-01-10,6,2,1,Supersix Evo Hi-Mod Team,Road - Elite Road - Carbon,10660,Louisville Race Equipment,"Louisville, KY"


# Data Wrangling

In [26]:
from typing import Any, Dict, List, Union

def separate(df: pd.DataFrame, 
             col: List[str], 
             into: List[str], 
             sep: List[str]) -> pd.DataFrame:
    
    df = df.copy()
    
    df[into] = df[col].str.split(sep, expand = True)
    df.drop(col, axis=1, inplace=True)
    
    return df


def str_replace_colnames(df: pd.DataFrame, 
                         pattern: str, 
                         replacement: str) -> pd.DataFrame:
    df = df.copy()
    
    df.columns = df.columns.str.replace(pattern, replacement)
    
    return df


def rearrange_cols(df: pd.DataFrame, 
                   cols: List[str]) -> pd.DataFrame:
    df = df.copy()
    
    assert(len(df.columns) == len(cols)), "cols length does not match the number of columns of the DataFrame"
    
    df = df[cols]
    
    return df


new_col_order = ['order_date',
                 'order_id',
                 'order_line',
                 'quantity',
                 'price', 
                 'total_price', 
                 'model', 
                 'bikeshop_name', 
                 'category_1', 
                 'category_2', 
                 'frame_material', 
                 'city', 
                 'state']

In [27]:
bike_orderlines_wrangled = bike_orderlines_joined.pipe(separate, 
                                                       'description', 
                                                       ["category_1","category_2","frame_material"], 
                                                       sep = " - ") \
                                                 .pipe(separate, 
                                                       'location', 
                                                       ["city", "state"], 
                                                       sep = ", ") \
                                                 .assign(total_price = bike_orderlines_joined['price'] * 
                                                                       bike_orderlines_joined['quantity']) \
                                                 .drop(['Unnamed: 0', 'customer.id', 'product.id'], axis=1) \
                                                 .pipe(str_replace_colnames, ".", "_") \
                                                 .pipe(rearrange_cols, new_col_order)

In [28]:
bike_orderlines_wrangled.head()

,order_date,order_id,order_line,quantity,price,total_price,model,bikeshop_name,category_1,category_2,frame_material,city,state
0,2011-01-07,1,1,1,6070,6070,Jekyll Carbon 2,Ithaca Mountain Climbers,Mountain,Over Mountain,Carbon,Ithaca,NY
1,2011-01-07,1,2,1,5970,5970,Trigger Carbon 2,Ithaca Mountain Climbers,Mountain,Over Mountain,Carbon,Ithaca,NY
2,2011-01-10,2,1,1,2770,2770,Beast of the East 1,Kansas City 29ers,Mountain,Trail,Aluminum,Kansas City,KS
3,2011-01-10,2,2,1,5970,5970,Trigger Carbon 2,Kansas City 29ers,Mountain,Over Mountain,Carbon,Kansas City,KS
4,2011-01-10,3,1,1,10660,10660,Supersix Evo Hi-Mod Team,Louisville Race Equipment,Road,Elite Road,Carbon,Louisville,KY
